In [ ]:
import numpy as np
import sympy as sp
import arm2fk as fk
import arm2vk as vk








theta0 = sp.symbols('theta0')
theta1 = sp.symbols('theta1')
theta2 = sp.symbols('theta2')

dh = sp.Matrix([[0, -.03825, 0, np.pi/2],
               [theta0-np.pi/2, 0, .170, -np.pi/2],
               [theta1, 0, .180, np.pi/2],
               [theta2, 0, .180, -np.pi/2]])

def dh2mat(dh):
    T = sp.eye(4)
    for i in range(dh.shape[0]):
        T = sp.MatMul(T, dhrow2mat(dh[i, :]))
    return T

def dhrow2mat(row):
    theta=row[0]
    d=row[1]
    a=row[2]
    alpha=row[3]
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta)*sp.cos(alpha), sp.sin(theta)*sp.sin(alpha), a*sp.cos(theta)],
                     [sp.sin(theta), sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha), a*sp.sin(theta)],
                     [0, sp.sin(alpha), sp.cos(alpha), d],
                     [0, 0, 0, 1]])
dhmat= dh2mat(dh)
dh2mat_lambdified = sp.lambdify([theta0, theta1, theta2], dhmat, 'numpy')
print(dhrow2mat(dh[0, :]))
print(dhrow2mat(dh[1, :]))
print(dhrow2mat(dh[2, :]))
print(dhrow2mat(dh[3, :]))
print(dh2mat_lambdified(0, 0, 0))

jacobian_dh2mat = sp.Matrix([
    dhmat.diff(theta0)[0:3, 3].T,
    dhmat.diff(theta1)[0:3, 3].T,
    dhmat.diff(theta2)[0:3, 3].T
]).T

jacobian_dh2mat_lambdified = sp.lambdify([theta0, theta1, theta2], jacobian_dh2mat, 'numpy')
print(jacobian_dh2mat_lambdified(0, 0, 0))


print("Matlab Screw FK",fk.arm2fk(0,0,0))
print("Matlab Jacobian",vk.arm2vk(0,0,0))


Matrix([[1, 0, 0, 0], [0, 6.12323399573677e-17, -1.00000000000000, 0], [0, 1.00000000000000, 6.12323399573677e-17, -0.0382500000000000], [0, 0, 0, 1]])
Matrix([[cos(theta0 - 1.5707963267949), -6.12323399573677e-17*sin(theta0 - 1.5707963267949), -1.0*sin(theta0 - 1.5707963267949), 0.17*cos(theta0 - 1.5707963267949)], [sin(theta0 - 1.5707963267949), 6.12323399573677e-17*cos(theta0 - 1.5707963267949), 1.0*cos(theta0 - 1.5707963267949), 0.17*sin(theta0 - 1.5707963267949)], [0, -1.00000000000000, 6.12323399573677e-17, 0], [0, 0, 0, 1]])
Matrix([[cos(theta1), -6.12323399573677e-17*sin(theta1), 1.0*sin(theta1), 0.18*cos(theta1)], [sin(theta1), 6.12323399573677e-17*cos(theta1), -1.0*cos(theta1), 0.18*sin(theta1)], [0, 1.00000000000000, 6.12323399573677e-17, 0], [0, 0, 0, 1]])
Matrix([[cos(theta2), -6.12323399573677e-17*sin(theta2), -1.0*sin(theta2), 0.18*cos(theta2)], [sin(theta2), 6.12323399573677e-17*cos(theta2), 1.0*cos(theta2), 0.18*sin(theta2)], [0, -1.00000000000000, 6.12323399573677e-17

In [ ]:
import time
def ik(x,y,z):
    # Define the desired position of the end effector
    x_d = x
    y_d = y
    z_d = z

    # Define the initial guess for the joint angles
    theta0_d = 0
    theta1_d = 0
    theta2_d = 0

    # Define the desired position of the end effector
    x_d = x
    y_d = y
    z_d = z

    # Define the initial guess for the joint angles
    theta0_d = 0
    theta1_d = 0
    theta2_d = 0

    # Define the number of iterations
    N = 100

    # Define the learning rate
    alpha = 0.01

    # Define the desired position of the end effector
    x_d = x
    y_d = y
    z_d = z

    # Define the initial guess for the joint angles
    theta0_d = np.arctan2(z_d,x_d)
    theta1_d = np.arccos(y_d/(.18+.18))
    theta2_d = 0

    # Define the number of iterations
    N = 2000

    # Define the learning rate
    alpha = 1

    #Compute the current position of the end effector
    x_c, y_c, z_c = fk.arm2fk(theta0_d, theta1_d, theta2_d)[0:3, 3]

    # Compute the error
    e = sp.Matrix([x_d - x_c, y_d - y_c, z_d - z_c])

    for i in range(N):
       
        
        # print(e)
        # Compute the Jacobian
        J = vk.arm2vk(theta0_d, theta1_d, theta2_d)
        J = np.linalg.pinv(J)
        # Update the joint angles
        theta0_d += (alpha * J[0, :].dot(e))[0]
        theta1_d += (alpha * J[1, :].dot(e))[0]
        theta2_d += (alpha * J[2, :].dot(e))[0]

        # Compute the current position of the end effector
        x_c, y_c, z_c = fk.arm2fk(theta0_d, theta1_d, theta2_d)[0:3, 3]

        # Compute the error
        e = sp.Matrix([x_d - x_c, y_d - y_c, z_d - z_c])

        if(e.norm() < 0.001):
            # print('early break, cycle: ',i)
            break
        
        

    return theta0_d, theta1_d, theta2_d ,i

cycles = np.array([])
execution_times = np.array([])

for n in range(2000):
    angs=np.random.uniform(-np.pi/3,np.pi/3,(3,1))
    pos=fk.arm2fk(angs[0],angs[1],angs[2])[0:3,3]
    start_time = time.time()
    ikangs=ik(pos[0],pos[1],pos[2])
    execution_time =time.time()-start_time
    cycles=np.append(cycles,ikangs[3])
    execution_times=np.append(execution_times,execution_time)

print("Mean cycles: ",np.mean(cycles))
print("Max cycles: ",np.max(cycles))
print("Min cycles: ",np.min(cycles))
print("Std cycles: ",np.std(cycles))
print("Median cycles: ",np.median(cycles))

print("Mean execution time (s): ",np.mean(execution_times))
print("Max execution time (s): ",np.max(execution_times))
print("Min execution time (s): ",np.min(execution_times))
print("Std execution time (s): ",np.std(execution_times))
print("Median execution time (s): ",np.median(execution_times))


Mean cycles:  17.3625
Max cycles:  146.0
Min cycles:  3.0
Std cycles:  14.514582107315388
Median cycles:  12.0
Mean execution time:  0.03757191395759583
Max execution time:  0.3000667095184326
Min execution time:  0.009002447128295898
Std execution time:  0.029801535942324122
Median execution time:  0.02700626850128174


In [ ]:
M = np.array([[0, 0, 1, 0],
              [0, 1, 0, 0],
              [-1, 0, 0, -.56825],
              [0, 0, 0, 1]])

S = np.array([[0, 1, 0],
              [-1, 0, -1],
              [0, 0, 0],
              [-.03825, 0, -.38825],
              [0, -.20825, 0],
              [0, 0, 0]])

def fkine(S,M,q):

    T = np.eye(4)
    for i in range(S.shape[1]):
        T = np.dot(T, twist2ht(S[:, i], q[i]))
    T = np.dot(T, M)
    return T

def twist2ht(twist, theta):
    twist = np.array(twist)
    omega = twist[0:3]
    v = twist[3:6]
    omega_hat = np.array([[0, -omega[2], omega[1]],
                          [omega[2], 0, -omega[0]],
                          [-omega[1], omega[0], 0]])
    R = np.eye(3) + np.sin(theta) * omega_hat + (1 - np.cos(theta)) * np.dot(omega_hat, omega_hat)
    p = np.dot((np.eye(3) * theta + (1 - np.cos(theta)) * omega_hat + (theta - np.sin(theta)) * np.dot(omega_hat, omega_hat)), v)
    T = np.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = p
    return T

def ikine(T):

    # print(T)
    # Set the bounds 
    lower_bound = -np.pi / 6 
    upper_bound = np.pi / 6 
    # # Generate a 3x1 random array within the specified bounds 
    q = (upper_bound - lower_bound) * np.random.rand(3) + lower_bound
    
     
    for i in range(1000000):
        # print(q)
        TG = fk.arm2fk(q[0],q[1],q[2])
        e = TG[0:3,3] - T[0:3,3]
        # print(T0)
        # print(e)
        # print(np.linalg.norm(e))
        if np.linalg.norm(e) < 1e-3:
            print("converged")
            print(q)
            print(TG)
            print(i)
            
            break
        J = vk.arm2vk(q[0],q[1],q[2]) #jacobian(S, q)

        # lambd = 0.1
        # dampedPseduoinv = np.linalg.pinv(J@J.T+lambd**2*np.eye(3))
        
        dq = np.linalg.pinv(J) @ np.vstack(e)
        q += dq.T[0]/1.5
        # print(q)
        
    
    
        
    return wrap_to_pi(q)



def wrap_to_pi(angle):
    return (angle + np.pi) % (2 * np.pi) - np.pi

def jacobian(S, q):
    J = np.zeros((6, S.shape[1]))
    T = np.eye(4)
    for i in range(S.shape[1]):
        T = T@twist2ht(S[:, i], q[i])
        Ad_T_Si = adjoint(T)@S[:, i]
        J[:, i] = Ad_T_Si
    return J

def adjoint(T): 
    """ Given a homogeneous transformation matrix T, calculate the corresponding 6x6 adjoint transformation matrix. """ 
    R = T[:3, :3] 
    p = T[:3, 3] 
    s = np.array([[0, -p[2], p[1]], 
                  [p[2], 0, -p[0]], 
                  [-p[1], p[0], 0]]) 
    AdT = np.block([[R, np.zeros((3, 3))],
                    [s @ R, R]]) 
    return AdT
            
def ht2twist(T):
    R = T[0:3, 0:3]
    p = T[0:3, 3]
    omega_hat = np.array([[0, -R[2, 1], R[1, 0]],
                          [R[2, 0], 0, -R[0, 2]],
                          [-R[1, 0], R[0, 1], 0]])
    omega = np.array([R[2, 1], R[0, 2], R[1, 0]])
    theta = np.arccos((np.trace(R) - 1) / 2)
    v = np.dot(np.linalg.inv(np.eye(3) * theta + (1 - np.cos(theta)) * omega_hat + (theta - np.sin(theta)) * np.dot(omega_hat, omega_hat)), p)
    twist = np.zeros(6)
    twist[0:3] = omega
    twist[3:6] = v
    return twist



def jacobe(S, M, q): 
    # Initialize the Jacobian matrix 
    J_b = np.zeros((6, len(q))) 
    # # Initialize the transformation matrix 
    T = np.eye(4) 
    B = S.copy() 
    # Loop through each joint 
    for i in range(len(q) - 1, -2, -1): 
        B[:, i] = twistspace2body(S[:, i], M) 
        # Update the transformation matrix 
        if i < len(q) - 1: 
            T = T @ twist2ht(B[:, i + 1], -q[i + 1]) 
            # Update the Jacobian matrix           
            J_b[:, i] = adjoint(T) @ B[:, i]
    return J_b

def skew(vector):
    """Create a skew-symmetric matrix from a 3-vector."""
    return np.array([[0, -vector[2], vector[1]],
                     [vector[2], 0, -vector[0]],
                     [-vector[1], vector[0], 0]])

def twistspace2body(V_s, T):
    """
    Convert a twist from the space frame to the body frame.
    
    Parameters:
    V_s -- Twist in the space frame (6x1 numpy array).
    T -- Homogeneous transformation matrix from the body frame to the space frame (4x4 numpy array).
    
    Returns:
    V_b -- Twist in the body frame (6x1 numpy array).
    """
    # T is the homogeneous transformation matrix from the body frame to the space frame
    R = T[:3, :3]  # Rotation matrix
    p = T[:3, 3]   # Position vector
    Adj_T = np.block([[R, skew(p) @ R], [np.zeros((3, 3)), R]])  # Adjoint representation of T
    V_b = Adj_T.T @ V_s
    return V_b


# q0=[np.pi/3,0,np.pi/6]
# T=fk.arm2fk(q0[0],q0[1],q0[2])
# print("Init angle",q0)
# print("Goal Pos",T)
# q=ikine(T)
# T2=fk.arm2fk(q[0],q[1],q[2])

# print("IK angles",q)
# print("IK angles Fk",T2)
# J=jacobian(S,[0,0,0])
# Je=jacobe(S,M,[0,0,0*np.pi/2])
# print(vk.arm2vk(q0[0],q0[1],q0[2]))
# print(Je)